# GDELT Crisis Analysis

The following notebook contains the code for the collection and pre-processing of data from the GDELT 2.0 Event Database by using Apache Spark. The resulting data is then provided via a Thrift Server to be accessed by Apache Superset for visualization and analysis. 
The example use case, which is implemented in this notebook, aims at visualizing global crises in a heatmap based on the average Goldstein Scale of the events, which occurred in the respective countries in a certain time period. As described in the provided documentation, the use case is implemented by following two different strategies:
- **Non-aggregated**: The data is collected, pre-processed and provided without prior aggregation.
- **Aggregated**: The data is collected, pre-processed and provided after aggregating the data by date and country.

It's important to note that in this notebook the different approaches are not strictly seperated, but are rather built on top of each other to optimize the performance. The reason for this is that the notebook is only intended for demonstration purposes, showcasing the different steps involved. Nevertheless, during testing, the two approaches are implemented and executed in a separate manner, to accurately measure the characteristics of both approaches. 

## Data Collection

The data is collected by downloading the CSV files containing the events from the GDELT 2.0 Event Database. This is necessary because the Event Database doesn't offer the possibility to export a complete dump of the data.

After the CSV files have been downloaded, they are then converted to Parquet files, to minimize the required storage space and to optimize the performance when reading the data into Apache Spark. 

The download and conversion process is executed in batches of months. This is done to minimize the storage space required at any given time, as every batch is compressed, which frees up storage space. Additionaly this allows for the simple addition of new months to already existing data.

In [1]:
import os

local_path = os.path.join(os.getcwd(), 'data')
parquet_path = os.path.join(local_path, 'parquet_main')

The time period for which the data should be downloaded is specified by the `start_date` and `end_date`.

If you want to change the time period, make sure to include full months, because the month will be skipped, if the notebook is executed again and the parquet file for the month already exists (e.g. with only half of the data)

We recommend keeping the default time period, to ensure fast execution of the notebook. 

In [2]:
from datetime import datetime, timedelta

# Choose time period for which to download the data
start_date = datetime.strptime('2021-12-01', '%Y-%m-%d')
end_date = datetime.strptime('2021-12-31', '%Y-%m-%d')

# Create a list of dates between start_date and end_date
date_list = [start_date + timedelta(days=x) for x in range(0, (end_date - start_date).days + 1)]

A list is created containing the download urls of all 15 minute interval CSV files for the specified time period.

In [3]:
import urllib.request

# Create a list containing download urls for each date
base_url = 'http://data.gdeltproject.org/gdeltv2/'
url_list = []
index = 0
url_list.append([])
month = date_list[0].month

# Create a nested list containing a list of months with the corresponding download urls
for date in date_list:
    if date.month != month:
        month = date.month
        index += 1
        url_list.append([])

    # Create the url and append it to the month list
    for x in range(0, 24):
        for y in range(0, 60, 15):
            date_tmp = date + timedelta(hours=x, minutes=y)
            url = base_url + date_tmp.strftime('%Y%m%d%H%M%S') + '.export.CSV.zip'
            url_list[index].append(url)

Creation of directories where the data will be stored.

In [4]:
# Create the local directory for the data if it doesn't exist
if not os.path.isdir(local_path):
    os.mkdir(local_path)
    
if not os.path.isdir(parquet_path):
    os.mkdir(parquet_path)

Intialization of the Spark Session to utilize Apache Spark for the conversion of the CSV files to Parquet files and later for the pre-processing of the data.

Hive support is enabled to allow running the Thrift Server to provide the data to Apache Superset.

The configuration for the Spark Session (e.g. the IP of the master) is stored in the `config` folder in the `spark_defaults.conf` file.

In [5]:
from pyspark.sql import SparkSession

# Start a spark session (see config folder for spark config)
spark = SparkSession.builder \
    .appName('Big Data Project') \
    .enableHiveSupport() \
    .getOrCreate()

The complete schema for the GDELT event data is defined to ensure that the CSV files are read correctly into Spark.

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType

# Define original data schema for csv files
schema = StructType([
    StructField("GlobalEventID", IntegerType(), True),
    StructField("Day", DateType(), True),
    StructField("MonthYear", IntegerType(), True),
    StructField("Year", IntegerType(), True),
    StructField("FractionDate", FloatType(), True),
    StructField("Actor1Code", StringType(), True),
    StructField("Actor1Name", StringType(), True),
    StructField("Actor1CountryCode", StringType(), True),
    StructField("Actor1KnownGroupCode", StringType(), True),
    StructField("Actor1EthnicCode", StringType(), True),
    StructField("Actor1Religion1Code", StringType(), True),
    StructField("Actor1Religion2Code", StringType(), True),
    StructField("Actor1Type1Code", StringType(), True),
    StructField("Actor1Type2Code", StringType(), True),
    StructField("Actor1Type3Code", StringType(), True),
    StructField("Actor2Code", StringType(), True),
    StructField("Actor2Name", StringType(), True),
    StructField("Actor2CountryCode", StringType(), True),
    StructField("Actor2KnownGroupCode", StringType(), True),
    StructField("Actor2EthnicCode", StringType(), True),
    StructField("Actor2Religion1Code", StringType(), True),
    StructField("Actor2Religion2Code", StringType(), True),
    StructField("Actor2Type1Code", StringType(), True),
    StructField("Actor2Type2Code", StringType(), True),
    StructField("Actor2Type3Code", StringType(), True),
    StructField("IsRootEvent", IntegerType(), True),
    StructField("EventCode", StringType(), True),
    StructField("EventBaseCode", StringType(), True),
    StructField("EventRootCode", StringType(), True),
    StructField("QuadClass", IntegerType(), True),
    StructField("GoldsteinScale", FloatType(), True),
    StructField("NumMentions", IntegerType(), True),
    StructField("NumSources", IntegerType(), True),
    StructField("NumArticles", IntegerType(), True),
    StructField("AvgTone", FloatType(), True),
    StructField("Actor1Geo_Type", IntegerType(), True),
    StructField("Actor1Geo_FullName", StringType(), True),
    StructField("Actor1Geo_CountryCode", StringType(), True),
    StructField("Actor1Geo_ADM1Code", StringType(), True),
    StructField("Actor1Geo_ADM2Code", StringType(), True),
    StructField("Actor1Geo_Lat", FloatType(), True),
    StructField("Actor1Geo_Long", FloatType(), True),
    StructField("Actor1Geo_FeatureID", StringType(), True),
    StructField("Actor2Geo_Type", IntegerType(), True),
    StructField("Actor2Geo_FullName", StringType(), True),
    StructField("Actor2Geo_CountryCode", StringType(), True),
    StructField("Actor2Geo_ADM1Code", StringType(), True),
    StructField("Actor2Geo_ADM2Code", StringType(), True),
    StructField("Actor2Geo_Lat", FloatType(), True),
    StructField("Actor2Geo_Long", FloatType(), True),
    StructField("Actor2Geo_FeatureID", StringType(), True),
    StructField("ActionGeo_Type", IntegerType(), True),
    StructField("ActionGeo_FullName", StringType(), True),
    StructField("ActionGeo_CountryCode", StringType(), True),
    StructField("ActionGeo_ADM1Code", StringType(), True),
    StructField("ActionGeo_ADM2Code", StringType(), True),
    StructField("ActionGeo_Lat", FloatType(), True),
    StructField("ActionGeo_Long", FloatType(), True),
    StructField("ActionGeo_FeatureID", StringType(), True),
    StructField("DATEADDED", StringType(), True),
    StructField("SOURCEURL", StringType(), True),
])

The following method is defined to download the CSV files from the GDELT website. 

Because the files are compressed, they are unzipped after being downloaded. The unzipped files are then deleted to free up storage space again.

In [7]:
import zipfile

def download_file(url):
    fname = url.split('/')[-1]
    folder_location = os.path.join(local_path, fname[:4], fname[4:6])

    # Download file from the specified url, if it doesn't exist yet
    if not os.path.isfile(os.path.join(folder_location, fname).replace(".zip", "")):
        try:
            urllib.request.urlretrieve(url, os.path.join(folder_location, fname))

            # Unzip zip file
            with zipfile.ZipFile(os.path.join(folder_location, fname), 'r') as zip_ref:
                zip_ref.extractall(folder_location)

            # Delete zip file
            os.remove(os.path.join(folder_location, fname))

        except Exception as e:
            print(f"An error occurred with file {fname}: {e}")

    else:
        print('File ' + fname + ' already exists')

The data is downloaded in batches of months. 

For each month the list of urls is split and distirbuted to multiple threads, which download and unzip the individual files in parallel, to reduce the time required for the download process. 

If there is already an existing parquet file for a month, the month is skipped, to prevent downloading the data again.

In [8]:
import shutil
from concurrent.futures import ThreadPoolExecutor

# Download files and write them to parquet files in parallel for each month
# This is done in batches to allow simple addition of new months to already existing data
for month_list in url_list:
    # Skip month if parquet file already exists
    if os.path.exists(os.path.join(parquet_path, month_list[0].split('/')[-1][:6] + ".parquet")):
        continue

    year_folder = os.path.join(local_path, month_list[0].split('/')[-1][:4])
    month_folder = os.path.join(year_folder, month_list[0].split('/')[-1][4:6])

    if not os.path.isdir(year_folder):
        os.mkdir(year_folder)

    if not os.path.isdir(month_folder):
        os.mkdir(month_folder)

    # Download all files from the url list in parallel (threads = no. processors on machine * 5)
    with ThreadPoolExecutor() as executor:
        executor.map(download_file, month_list)

    # Read all csv files of one month into a spark dataframe
    df = spark.read.csv(month_folder, sep='\t', header=False, schema=schema, dateFormat='yyyyMMdd')

    # Write the data of one month into a parquet file
    df.write.parquet(os.path.join(parquet_path, month_list[0].split('/')[-1][:6] + ".parquet"), mode='overwrite')

    # Delete the csv files to free up disk space
    shutil.rmtree(month_folder)

## Non-aggregated Approach

The non-aggregated approach loads the data, pre-processes it and provides it to be used by Superset. 

The necessary aggregation of the avereage Goldstein Scale per country over a specific time period is triggered by the SQL queries, which are sent from Superset to the Thrift Server and are then processed by Spark. Therefore the aggregation is done on-demand and must be re-calculated every time.

All parquet files, which are stored in the `parquet_main` directory, are loaded into a Spark dataframe. 

In [9]:
# Load all parquet files from the data directory into spark
df_base = spark.read.parquet(parquet_path + '/*.parquet')

The country codes in the GDELT data are chosen based on the FIPS10-4 standard. To be able to visualize the data in Apache Superset, the country codes need to be mapped to ISO 3166-1 alpha-2 country codes, because Superset uses these codes to identify the countries and display them on the heatmap.

A CSV file containing the mapping is loaded into Spark. The country codes are then mapped by joining the GDELT dataframe with the mapping dataframe.

To boost the performance of the expensive operation, the mapping dataframe is broadcasted, which means that the dataframe is copied to each worker node in the cluster. This is possible because the mapping dataframe is small in size.

In [10]:
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as F

# CSV file containing a mapping from FIPS10-4 country codes to ISO 3166-1 alpha-2 country codes (necessary for superset heatmap)
mapping_file_path = os.path.join(os.getcwd(), 'util', 'country_code_mapping.csv')

# Load mapping file outside of spark (small dataset)
df_mapping = spark.read.csv(mapping_file_path, sep=';', header=True, inferSchema=True).select(
    F.col('FIPS 10-4'),
    F.col('ISO 3166-1')
)

# Map the country codes
df_non_aggregated = df_base.join(broadcast(df_mapping), df_base['ActionGeo_CountryCode'] == df_mapping['FIPS 10-4'],
                                 'left_outer')

df_non_aggregated = df_non_aggregated \
    .withColumn('FIPS 10-4', F.col('ActionGeo_CountryCode')) \
    .withColumn('ActionGeo_CountryCode', F.col('ISO 3166-1')) \
    .drop('ISO 3166-1')

The dataframe is then cached to prevent re-doing the data loading and all of the previous operations. The data is primarily cached in memory, but spills over to disk if not enough memory space is available.

Due to Spark's lazy evaluation, a count operation is executed to trigger the caching of the dataframe.

In [11]:
# Cache the dataframe to prevent re-doing data loading & country code mapping
df_non_aggregated.cache()

# Load data & trigger caching
event_count = df_non_aggregated.count()

print("Total number of events:", event_count)

Total number of events: 2954730


The resulting dataframe is registered as a global temporary view, which assigns the specified name to the logical plan, which describes how the dataframe is created. Because the underlying data is cached, the transformation steps in the logical plan don't have to be re-executed every time the view is accessed.

Subsequently, SQL queries sent to the Thrift Server can access the cached dataframe by using the specified name, in this case "global_temp.GDELT".

The data is not materialized on disk. Instead the data is kept in memory, which is why lifetime of the view is also bound to the lifetime of the Spark application.

In [12]:
# Virtual table which can be accessed by the thrift server
df_non_aggregated.createOrReplaceGlobalTempView("GDELT")

The following code checks, for which FIPS 10-4 country codes in the dataset there is no corresponding ISO 3166-1 alpha-2 country code and counts how many events are affected by this.
For example the following FIPS 10-4 country codes don't have a corresponding ISO 3166-1 alpha-2 country code:
- PF (Paracel Islands)
- NT (Netherlands Antilles)
- PG (Spratly Islands)
- ...

The defined SQL query, which is sent to Spark to aggregate the data on-demand to visualize the result in Superset, ignores these events, as they can't be visualized on the heatmap.

In [13]:
# Check for country codes where there is no coresponing ISO 3166-1 alpha-2 country code
df_non_aggregated.filter((F.col('ActionGeo_CountryCode').isNull()) & (F.col('FIPS 10-4').isNotNull())) \
    .groupBy('FIPS 10-4') \
    .agg(F.count('*').alias('EventCount')) \
    .sort('EventCount', ascending=False) \
    .show()

+---------+----------+
|FIPS 10-4|EventCount|
+---------+----------+
|       OS|      1501|
|       RB|      1300|
|       OC|       321|
|       YI|        48|
|       WQ|        27|
|       NT|         8|
|       LQ|         7|
|       PG|         5|
|       JN|         1|
|       PF|         1|
|       CR|         1|
|       JQ|         1|
+---------+----------+


## Aggregated Approach

The aggregated approach loads the data, selects only the relevant columns, pre-processes it and pre-aggregates it before providing it to be used by Superset.

The pre-aggregation calculates the sum of the Goldstein Scale values and the number of events for each country per day, so the average can still be calculated for a specific time period dynamically. 

The caluclation of the average Goldstein Scale per country over a specific time period is then triggered by the SQL query, which is sent from Superset to the Thrift Server. Spark calculates the average Goldstein Scale by utilizing the pre-aggregated data, which reduces the amount of data that needs to be processed when the data is visualized in Superset.

Only the necessary columns are selected from the dataframe to reduce the amount of data that needs to be processed. These columns include the date, the country code and the Goldstein Scale value.

This is done using the datframe, in which the country codes are already mapped to ISO 3166-1 alpha-2 country codes, to reduce the execution time of this notebook. In the tests, the mapping of country codes is done separately after the data has been aggregated already, to ensure that the performance of the two approaches can be compared accurately.

In [14]:
# Select only relevant columns for the aggregation
df_selection = df_non_aggregated.select(
    F.col('Day'),
    F.col('ActionGeo_CountryCode'),
    F.col('GoldsteinScale')
)

The number of null values in each of the relevant columns are shown to check if there are any null values in the dataset.

In [15]:
# Number of null values in each column
df_selection.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_selection.columns]).show()

+---+---------------------+--------------+
|Day|ActionGeo_CountryCode|GoldsteinScale|
+---+---------------------+--------------+
|  0|                88375|            49|
+---+---------------------+--------------+


The rows with null values are dropped, as they can't be used for the aggregation.

In [16]:
# Drop rows with null values as they would distort the aggregation
df_selection = df_selection.na.drop()

# Number of rows not considering null values
event_count_without_null = df_selection.count()

print("Events removed from dataset:", event_count - event_count_without_null)

Events removed from dataset: 88424


The sum of the Goldstein Scale values and the number of events for each country per day are calculated.

This pre-aggregation allows the average Goldstein Scale to be calculated dynamically for a specific time period, while reducing the amount of data that needs to be processed significantly.

In [17]:
# Aggregate the values by date and country so there is only one value per country per day
df_aggregated = df_selection.groupBy('Day', 'ActionGeo_CountryCode').agg(
    F.sum('GoldsteinScale').alias('GoldsteinScaleSum'),
    F.count('*').alias('EventCount')
)

The result of the aggregation is then cached to prevent re-doing the loading, pre-processing and aggregation every time an SQL query is sent to the Thrift Server.

Again, the caching is triggered by executing a count operation. 

In [18]:
# Cache the dataframe to prevent re-doing the aggregation
df_aggregated.cache()

# Trigger caching of the final aggregated dataframe
aggregation_count = df_aggregated.count()

print("Number of rows in aggregation:", aggregation_count)

Number of rows in aggregation: 11268


The resulting dataframe is also registered as a global temporary view, so it can be accessed through the Thrift Server by using the specified name "global_temp.GDELT_AGGR".

In [19]:
# Virtual table which can be accessed by the thrift server
df_aggregated.createOrReplaceGlobalTempView("GDELT_AGGR")

As last step, the Thrift Server is started. The Thrift Server provides an interface for the execution of SQL queries on the Spark data using JDBC/ODBC. Therefore the data in Spark can be accessed by clients (e.g. Apache Superset) using SQL queries. The queries are then processed by Spark, which acts as a distributed SQL query engine.

Normally the Thrift Server would be started by running the `start-thriftserver.sh` script in the `sbin` directory of the Spark installation. For the given use case, this is not a suitable approach, because the Thrift Server would be started in a separate process, without access to the context of the Spark application. Thereby the Thrift Server wouldn't be able to access the unmaterilized datasets, which are registered as global temporary views.

The following workaround to this problem has been identified and was implemented in this project. The Thrift Server is started by utilizing the Py4J library, which allows the execution of Java code from Python. The Java class of the Thrift Server `HiveThriftServer2` is imported and the main method is called using the JVM instance of the Spark context. Consequently, the Thrift Server is started in the same JVM as the Spark application and uses the same Spark context, which allows the Thrift Server to access the registered global temporary views. As a result the cached data can be accessed by clients using SQL queries.

In [20]:
from py4j.java_gateway import java_import

# Retrieve the spark context from the current spark session
sc = spark.sparkContext

# Import the HiveThriftServer2 class using the JVM instance of the spark context
java_import(sc._jvm, "org.apache.spark.sql.hive.thriftserver.HiveThriftServer2")

# Dummy java arguments for main method
java_args = sc._gateway.new_array(sc._gateway.jvm.java.lang.String, 0)

# Start the thrift server by calling the main method of the imported class
sc._jvm.org.apache.spark.sql.hive.thriftserver.HiveThriftServer2.main(java_args)